# Design methods: Contexts, modules, functions

For verification purposes, a reusable construct that is proven valid is most worthy in complex projects. Therefore a classic HDL developer would write a configureable module and integrate that into his hierarchy.

VHDL, for instance, supports function calls, but does not distinguish between synthesizeable and simulation constructs. Here, the story is different: Function calls can just execute and return a result, or they can yield hardware elements (the generator way) that are synthesizeable.

In [1]:
from cyhdl import *

## RTL functions

We can create a hardware generator function in **cyhdl** notation. Its differences from a `@block`:
   * It will not infer into a module, thus, has no interface
   * It can not contain a process (`@always..`)
   * It does not allow exception type statements such as `assert` and `raise` in most contexts. Avoid.

Unlike a `@hdlmacro`, it can contain conditional (if..else) statements. It is therefore context sensitive which is represented by the `rtl` parameter. It is mandatory that the first argument is spelled `rtl`.

Due to reasons with translation, it is currently necessary to wrap such a decorated function inside a class. This is not a bad idea anyhow, to create some structure:

In [2]:
class primitives:
    @rtl_function
    def muxer(rtl, EN, A, B, Q):
        if EN:
            Q.next = A
        else:
            Q.next = B

We could also consider using a `@hdlmacro` for pure HDL emission, however there's a catch:

* IRL notation requirement: We must return structures using yield using IRL elements
* We would have to determine the `If` architecture context beforehand or pass it in a parameter (such as `self`).

The `@rtl_function` does the selection for us at elaboration or evaluation time. Its internal logic generator inherits the architecture from its calling process.

Note:

* A `@rtl_function` is an **unbound** function that is internally passed a `rtl` context.
Therefore, the `primitive` class is not instanced, but merely a container structure.
* It **prohibits** access to global definitions. Everything you use inside this function
  must be either taken from the `rtl` context (normally, the calling process) or passed as argument.
* Do not forget to use `rtl` as first argument. You can not use 'self' or another notation.
* It is a dynamic dual use function that is possibly transpiled to IRL, depending on the context

A test unit making use of this construct may look as follows:

In [3]:
@block
def unit1(clk : ClkSignal, a : Signal, b : Signal, en : Signal.Type(bool),
          q : Signal.Output, MODE : int):
    
    # This is a simple expression that can be assigned to a signal:
    expr_and = a & b
    expr_or = a | b
    # This results in a logic generator, where the result
    # is a parameter:
    
    logic = []
    
    if MODE == 1:
        logic += [ q @assign@ expr_and ]
    elif MODE == 2:
        logic = [ q @assign@ expr_or ]
    else:
        @always(clk.posedge)
        def ff():
            primitives.muxer(en, a, b, q)
        
        logic += [ ff ]

    return logic

We note a few things:
   * The `@rtl_function` funclet does not allow us to specify a process element
   * The hardware generator function is called **from within** an `@always` process.

Let's create a dummy target context for this module to evaluate the `@hdlmacro` in.
Then we instance a few signals:

In [4]:
from myirl.targets.dummy import DummyTargetModule
context = DummyTargetModule()

a, b, q = [ Signal(intbv()[8:], name = n) for n in "abc" ]
en = Signal(bool(), name = 'en')

We create a local `muxer` instance, initialize the signals and evaluate each of the actions manually.  Note two things:
* The muxer is created in the first line, but no decision on the context was made yet
* During `.evaluate()`, the function is actually called with the required context.

In [5]:
m = primitives.muxer(en, a, b, q)
m.evaluate(context)

After evaluation, we can actually inspect the true Python code behind the function.
This is only valid, if we call it from a hardware generation context.

In [6]:
print(m.unparse())



class __wrapper_muxer__():

    @rtl_function
    def muxer(rtl, EN, A, B, Q):
        (yield [rtl.If(EN).Then(Q.set(A)).Else(Q.set(B))])



We cycle through the possible `en` values (only digital '0' and '1' for Bool signals) and check the result:

In [7]:
for val, res in (True, a), (False, b):
    init = [ en.set(val), a.set(1), b.set(2) ]
    [ i.evaluate() for i in init ]
    m.evaluate(context)
    assert q.evaluate() == res.evaluate()
    print(q.evaluate())

01
02


Using this technique, we evaluate if the muxer function is doing the right thing for our signal type. The same should then happen in the hardware and corresponding simulation.

## Creating a test bench



In [8]:
from myirl.test.icarus import ICARUS
from cyrite.simulation import sim

@sim.testbench(ICARUS, 'ns')
@block
def tb_unit1():
    a, b, q = [ Signal(intbv()[8:]) for _ in range(3) ]
    en = Signal(bool())
    clk = ClkSignal(name = 'clk')
    
    uut = unit1(clk = clk, en = en, a = a, b = b, q = q, MODE = 3)
    
    @always(delay(1))
    def clkgen():
        clk.next = ~clk
    
    @sequence
    def main():
        """Yes, we call the main stimulation routine 'main' to
denote the sequence creating a wave trace"""
        en.next = False
        yield delay(1)
        a.next = 12
        b.next = 4
        en.next = True
        for i in range(4):
            yield delay(1)
            # assert q == 4
            yield clk.posedge
            en.next = ~en
        # assert q == 12
        
        yield delay(20)
        
        raise StopSimulation
        
    return instances()


## Running the test bench

In [9]:
tb = tb_unit1()
tb.run(80)

 Writing 'unit1' to file /tmp/myirl_tb_unit1_p2x0f1ui/unit1.v 
DEBUG NAME q <class 'myirl.library.shift.SAlias'>
 Writing 'tb_unit1' to file /tmp/myirl_tb_unit1_p2x0f1ui/tb_unit1.v 
 Creating library file module_defs.v 


0

### Displaying the trace

In [10]:
from cyrite import waveutils
waveutils.draw_wavetrace(tb, 'tb_unit1.vcd', 'clk')

### RTL specific signals

A `@rtl_function` may create internal auxiliary signals that are also depending on the target architecture.

In [11]:
class primi_x(primitives):
    @rtl_function
    def muxer2(rtl, EN, A, B, Q, wire):
        q = rtl.Signal(rtl.intbv()[8:])
        
        if EN:
            q.next = A
        else:
            q.next = B
            
        Q.next = q
        

Unlike a sequential process that would create a variable, a `@rtl_function` allows to create an internal signal, as if it was outside the process.

However, there are dragons:
* When called from a clock synchronous process, `Q` will receive the value of `q` with a delay of one clock period, so this is not a continuous assignment. This is a feature, not a bug!

In [12]:
@block
def tb(muxer):
    a, b, q = [ Signal(intbv()[8:]) for _ in range(3) ]
    en = Signal(bool())
    clk = ClkSignal(name = 'clk')
    
    @always(clk.posedge)
    def ff():
        muxer(en, a, b, q, intbv)
           
    ff.intbv = intbv
    return instances()

In [13]:
t = tb(primi_x.muxer2)

/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/components.py:118: UserWarning: Fallback: Pass through other argument for arg muxer (<class 'function'>)
  base.warnings.warn(msg)


In [14]:
a, b, q = [ Signal(intbv()[8:]) for _ in range(3) ]
en = Signal(bool())
clk = ClkSignal(name = 'clk')

context.Signal = Signal

# Examine translated IRL:
m = primi_x.muxer2(en, a, b, q, intbv)
m.this = context
m._run(context)
print(m.unparse())



class __wrapper_muxer2__():

    @rtl_function
    def muxer2(rtl, EN, A, B, Q, wire):
        q = rtl.Signal(rtl.intbv()[8:])
        (yield [rtl.If(EN).Then(q.set(A)).Else(q.set(B)), Q.set(q)])



In [15]:
f = t.elab(targets.VHDL)

 Writing 'tb' to file /tmp/myirl_tb_gbq1gzfy/tb.vhdl 


In [16]:
!cat {f[0]}

-- File generated from source:
--     /tmp/ipykernel_915/235411190.py
-- (c) 2016-2022 section5.ch
-- Modifications may be lost, edit the source file instead.

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.txt_util.all;
use work.myirl_conversion.all;

entity tb is
end entity tb;

architecture cyriteHDL of tb is
    -- Local type declarations
    -- Signal declarations
    signal s_19c7 : unsigned(7 downto 0);
    signal q : unsigned(7 downto 0);
    signal b : unsigned(7 downto 0);
    signal a : unsigned(7 downto 0);
    signal en : std_ulogic;
begin
    
ff:
    process(clk)
    begin
        if rising_edge(clk) then
            if (en = '1') then
                s_19c7 <= a;
            else
                s_19c7 <= b;
            end if;
            q <= s_19c7;
        end if;
    end process;
end architecture cyriteHDL;



## cyrite_factory design classes

For a configureable top level design, a structure is required that allows to:

* Verify a design configuration with various simulator backends
* Output the design to a HDL
* Directly synthesize the design via yosys
* Perform these steps in various ways for different architectures

Classical design methods often rely on external tcl scripts to keep configuration separate from source. A `@cyrite_factory` Module class however tries to keep it all in one place or uses derivation strategies for different configurations.

### Example

First, we create an architecture class for a specific target, in this case, the GHDL simulator.

In [17]:
from cyhdl import *
from myirl.library.architecture import Architecture
from cyrite.simulation.ghdl import GHDL

class MyArchitecture(Architecture):
    def __init__(self, target = GHDL):
        self.wiretype = intbv
        self.sim_class = target
        self.target_class = target._default_target 

class MyDesign(cyrite_factory.Module):
    
    def __init__(self, n : int, arch = MyArchitecture()):
        self.sigsize = n
        super().__init__("my_design", arch)
    
    @cyrite_factory.block_component
    def my_top(self,
               clk : ClkSignal, en : Signal.Type(bool),
               a : Signal, q : Signal.Output):
        
        @always(clk.posedge)
        def worker():
            if en:
                q.next = a
                
        return instances()
    
    def build(self, *args, **kwargs):
        clk = self.ClkSignal()
        a, b = [ self.Signal(intbv()[self.sigsize:]) for _ in range(2) ]
        en = Signal(bool())
        uut = self.my_top(clk, en, a, b)
        target = self.target_class()
        files = self.elab(target)
        print("Build the entire design", files)

The general strategy using a design class is to create a general configuration first, then generate a hardware hierarchy from it.

For instance, building a set of VHDL files:

In [18]:
design = MyDesign(12)
design.build()

 Declare obj 'my_top' in context '(MyDesign 'my_design')'(<class '__main__.MyDesign'>) 
 Writing 'my_top' to file /tmp/myirl_my_design_z5qvqn_w/my_top.vhdl 
 Creating library file module_defs.vhdl 
Build the entire design ['/tmp/myirl_my_design_z5qvqn_w/my_top.vhdl', 'module_defs.vhdl']


By inheritance, we can create variants of this module, for instance:
* Create design where the hardware design is shipped as compiled simulation, with the test bench available as source
* Create a design that is portable between several FPGA targets and simulators

### Thumb rules

The question might come up when to use `@cyrite_factory` classes with `@cyrite_factory.block_component` methods instead of `@block` functions. The extra complexity of a class is then justified, when designs are derived with a different functionality on the top level.
Otherwise, use a `@block` unit is whenever possible for reusable logic. It can be configured using passthrough options at instance time, or it can emit parametrizeable interfaces to some extent using the `ParametricSignal` signal type from `myirl.library.parametric`, see also [Parametric Signals](signals_interfaces.ipynb#Parametric-signals).